# Dragonfly Algorithm for CNN Hyperparameter Tuning: Soil Image Classification

## 🧠 Project Overview

This project utilizes the **Dragonfly Algorithm (DA)** to optimize hyperparameters of a **Convolutional Neural Network (CNN)** for **soil image classification**. The Dragonfly Algorithm is inspired by the static and dynamic swarming behaviors of dragonflies in nature and has proven effective in solving complex optimization tasks.

---

## 🎯 Objectives

- Employ the Dragonfly Algorithm to fine-tune CNN hyperparameters.
- Improve the CNN model’s accuracy in classifying soil image types.
- Benchmark performance against other metaheuristics like Hybrid PSO, WOA, and ALO.

---

## 🛠️ Methodology

### 1. Dataset
- Dataset includes soil images categorized into multiple classes.
- Preprocessing steps: image resizing, normalization, label encoding.
- Split into **training** and **testing** sets.

### 2. CNN Architecture
- Two convolutional layers with ReLU activation and max-pooling.
- Dense layer followed by dropout for regularization.
- Output layer uses Softmax for multi-class prediction.

### 3. Hyperparameters Optimized
- **Learning Rate**
- **Number of Filters**
- **Dropout Rate**

### 4. Dragonfly Algorithm (DA)
- Mimics static and dynamic swarming behaviors.
- Dragonflies search for optimal hyperparameters by updating positions influenced by:
  - **Separation** (avoidance of crowding),
  - **Alignment** (velocity matching with neighbors),
  - **Cohesion** (attraction toward the center of the neighborhood),
  - **Attraction to Food Source** (best solution),
  - **Distraction from Enemy** (worst solution).

- Fitness is calculated using CNN validation accuracy after limited training epochs.
- The dragonfly with the highest fitness is selected to train the final model.

---

## 📈 Evaluation Metrics

- **Test Accuracy**: Measures prediction success on unseen test images.
- **Confusion Matrix**: Visualizes prediction accuracy across all classes.
- **ROC Curve (Optional)**: Illustrates performance trade-offs, especially for binary classes.

---

## 📊 Final Results

- **Best Learning Rate**: *e.g., 0.0010*
- **Best Number of Filters**: *e.g., 32*
- **Best Dropout Rate**: *e.g., 0.25*
- **Test Accuracy**: *e.g., 83.7%*

> The Dragonfly Algorithm effectively improved CNN accuracy by guiding the model toward more optimal hyperparameter combinations.

---

## 📌 Conclusion

- The Dragonfly Algorithm is a competitive method for CNN hyperparameter tuning.
- It offers strong performance on soil classification tasks.
- DA adapts well to complex landscapes and high-dimensional search spaces.

---

## 🔭 Future Work

- Expand tuning to more CNN parameters like number of layers and batch size.
- Explore hybrid approaches combining DA with PSO or Genetic Algorithms.
- Apply DA to other domains like medical imaging, satellite classification, etc.

---

## 📁 Repository Structure



In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Set image size and batch size
img_height, img_width = 256,256
batch_size = 64

# Paths to your data
train_dir = 'ProcessedSpilt_dataset/train'
val_dir = 'ProcessedSpilt_dataset/val'
test_dir = 'ProcessedSpilt_dataset/test'

# Initialize data generators
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
val_generator = datagen.flow_from_directory(val_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
test_generator = datagen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical', shuffle=False)

num_classes = len(train_generator.class_indices)

Found 2240 images belonging to 5 classes.
Found 480 images belonging to 5 classes.
Found 480 images belonging to 5 classes.


In [4]:
# === Fitness Function ===
def evaluate_model(params):
    lr, f1, f2, dense = params
    f1, f2, dense = int(f1), int(f2), int(dense)

    model = models.Sequential([
        layers.Input(shape=(img_height, img_width, 3)),
        layers.Conv2D(f1, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        layers.Conv2D(f2, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        layers.Conv2D(f2*2, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),

        layers.Flatten(),
        layers.Dense(dense, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=0)
    early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

    history = model.fit(train_generator, 
                        validation_data=val_generator, 
                        epochs=10, 
                        verbose=0, 
                        callbacks=[lr_scheduler, early_stop])
    return history.history['val_accuracy'][-1]

In [5]:
# === Dragonfly Optimization ===
def dragonfly_optimize(bounds, dim=4, pop_size=6, iterations=5):
    positions = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], size=(pop_size, dim))
    best_score = -np.inf
    best_position = None
    convergence = []

    for iter in range(iterations):
        scores = []
        for i in range(pop_size):
            fitness = evaluate_model(positions[i])
            scores.append(fitness)
            if fitness > best_score:
                best_score = fitness
                best_position = positions[i].copy()
        convergence.append(best_score)
        print(f"Iteration {iter+1}, Best Accuracy: {best_score:.4f}")
        
        for i in range(pop_size):
            positions[i] += np.random.uniform(-0.05, 0.05, dim)
            positions[i] = np.clip(positions[i], [b[0] for b in bounds], [b[1] for b in bounds])

    return best_position, best_score, convergence

In [6]:
bounds = [
    [0.00005, 0.001],   # learning rate (slightly smaller upper bound)
    [64, 128],          # conv1 filters
    [128, 256],         # conv2 filters
    [256, 512]          # dense units
]
best_params, best_score, convergence = dragonfly_optimize(bounds) 

2025-05-24 18:10:52.626204: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-05-24 18:10:52.626244: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-05-24 18:10:52.626249: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-05-24 18:10:52.626272: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-24 18:10:52.626290: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/simarkalsi/Cd Project Soil/soil/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `supe

Iteration 1, Best Accuracy: 0.4000


KeyboardInterrupt: 

In [6]:
# === Final Training ===
lr, f1, f2, dense = best_params
f1, f2, dense = int(f1), int(f2), int(dense)

final_model = models.Sequential([
    layers.Conv2D(f1, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(f2, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(f2*2, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(dense, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

final_model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)

history = final_model.fit(train_generator, 
                          validation_data=val_generator,
                          epochs=25,
                          verbose=1,
                          callbacks=[early_stop, lr_scheduler])

/Users/simarkalsi/Cd Project Soil/soil/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 120s 686ms/step - accuracy: 0.6308 - loss: 10.7043 - val_accuracy: 0.2103 - val_loss: 18.0857 - learning_rate: 9.1184e-05
Epoch 2/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 117s 680ms/step - accuracy: 0.7736 - loss: 9.9403 - val_accuracy: 0.4359 - val_loss: 25.9603 - learning_rate: 9.1184e-05
Epoch 3/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 117s 685ms/step - accuracy: 0.8200 - loss: 10.5384 - val_accuracy: 0.7966 - val_loss: 8.0468 - learning_rate: 9.1184e-05
Epoch 4/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 117s 683ms/step - accuracy: 0.8448 - loss: 9.4066 - val_accuracy: 0.8675 - val_loss: 6.2227 - learning_rate: 9.1184e-05
Epoch 5/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 118s 687ms/step - accuracy: 0.8601 - loss: 10.7554 - val_accuracy: 0.8513 - val_loss: 11.5986 - learning_rate: 9.1184e-05
Epoch 6/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 118s 687ms/step - accuracy: 0.8827 - loss: 8.3663 - val_accuracy: 0.9265 - val_loss: 2.5664 - learning_rate: 9.1184e-05
Epoch 7/25
171/171 ━━━━━━━━━━━━━━━

In [7]:
# Evaluate on test set
test_loss, test_acc = final_model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 551ms/step - accuracy: 0.9767 - loss: 1.6862
Test Accuracy: 0.9744


In [8]:
final_model.save("dragonfly97.keras")